In [ ]:
library(shiny)
library(datasets)
library(ggplot2)
library(shinythemes)

ui <- shinyUI(fluidPage(theme  = "mytheme.css",
  titlePanel("Data Analysis"),
  tabsetPanel(
    tabPanel("Upload File",
             titlePanel("Uploading Files"),
             sidebarLayout(
               sidebarPanel(
                 fileInput('file1', 'Choose CSV File',
                           accept=c('text/csv', 
                                    'text/comma-separated-values,text/plain', 
                                    '.csv')),
                 
             
                 tags$br(),
                 checkboxInput('header', 'Header', TRUE),
                 radioButtons('sep', 'Separator',
                              c(Comma=',',
                                Semicolon=';',
                                Tab='\t'),
                              ','),
                 radioButtons('quote', 'Quote',
                              c(None='',
                                'Double Quote'='"',
                                'Single Quote'="'"),
                              '"')
                 
               ),
               mainPanel(
                 tableOutput('contents')
               )
             )
    ),
    tabPanel("Graphs",
             pageWithSidebar(
               headerPanel('Scatter Plot'),
               sidebarPanel(
                 
                 # "Empty inputs" - they will be updated after the data is uploaded
                 selectInput('xcol', 'X Variable', "",selected = 1),
                 selectInput('ycol', 'Y Variable', "", selected = 1)
                 
               ),
               mainPanel(
                 plotOutput('MyPlot')
               )
             )
    ),
    tabPanel("Regression Analysis",
             pageWithSidebar(
               headerPanel(''),
               sidebarPanel(),
               mainPanel(
                 tabsetPanel(type = "tabs",
                             
                             tabPanel("Scatterplot", plotOutput("scatterplot")), # Plot
                             tabPanel("Distribution", # Plots of distributions
                                      fluidRow(
                                        column(6, plotOutput("distribution1")),
                                        column(6, plotOutput("distribution2")))
                             ),
                             tabPanel("Model Summary", verbatimTextOutput("summary")) # Regression output
                            
                             
                 )
               )
             ))
    
  )
)
)

server <- shinyServer(function(input, output, session) {
  # added "session" because updateSelectInput requires it
  
  
 data <- reactive({ 
    req(input$file1) ## ?req #  require that the input is available

    inFile <- input$file1 

    # tested with a following dataset: write.csv(mtcars, "mtcars.csv")
    # and                              write.csv(iris, "iris.csv")
    df <- read.csv(inFile$datapath, header = input$header, sep = input$sep,
             quote = input$quote)
    
    # Update inputs (you could create an observer with both updateSel...)
    # You can also constraint your choices. If you wanted select only numeric
    # variables you could set "choices = sapply(df, is.numeric)"
    # It depends on what do you want to do later on.
    
    updateSelectInput(session, inputId = 'xcol', label = 'X Variable',
                      choices = names(df), selected = names(df)[2])
    updateSelectInput(session, inputId = 'ycol', label = 'Y Variable',
                      choices = names(df), selected = names(df)[2])
    
    return(df)
  })
  
  output$contents <- renderTable({
    data()
  })
  
  output$MyPlot <- renderPlot({
    # for a histogram: remove the second variable (it has to be numeric as well):
    # x    <- data()[, c(input$xcol, input$ycol)]
    # bins <- nrow(data())
    # hist(x, breaks = bins, col = 'darkgray', border = 'white')
    
    # Correct way:
    # x    <- data()[, input$xcol]
    # bins <- nrow(data())
    # hist(x, breaks = bins, col = 'darkgray', border = 'white')
    
    
    # I Since you have two inputs I decided to make a scatterplot
    ggplot(data(), aes_string(x = input$xcol, y = input$ycol)) +
      geom_point()
   
    
  })
    
    
  output$summary <- renderPrint({
    fit <- lm(data()[,input$xcol] ~ data()[,input$ycol])
    names(fit$coefficients) <- c("Intercept", input$var2)
    summary(fit)
  })
  
  output$scatterplot <- renderPlot({
    plot(data()[,input$xcol], data()[,input$ycol], main="Scatterplot",
         xlab=input$xcol, ylab=input$ycol, pch=19)
    abline(lm(data()[,input$xcol] ~ data()[,input$ycol]), col="red")
    lines(lowess(data()[,input$xcol],data()[,input$ycol]), col="blue")
  }, height=400)
  
  output$distribution1 <- renderPlot({
    hist(data()[,input$xcol], main="", xlab=input$xcol)
  }, height=300, width=300)
  
  # Histogram output var 2
  output$distribution2 <- renderPlot({
    barplot(data()[,input$ycol], main="", xlab=input$ycol)
  }, height=300, width=300)

})

shinyApp(ui, server)